In [1]:
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook as tqdm
from nltk.metrics.distance import jaccard_distance, edit_distance, masi_distance

from typing import Dict, Tuple
import spacy
from spacy.lang.pt.examples import sentences 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

In [2]:
nlp = spacy.load('pt_core_news_sm')
doc = ET.parse('./Project1/KB.xml')

## Construct the dataset
* As there are some questions that repeat and those map to different answers at the same time, we decided to keep only the first question -> answer mapping

In [3]:
data = []
reference: Dict[str, Tuple[int, str]] = {}

for document in doc.findall('documento'):
    for faq in document.find('faq_list').findall('faq'):
        answer = faq.find('resposta')
        idx = answer.attrib['id']
        answer = answer.text.strip()
        
        for question in faq.find('perguntas').findall('pergunta'):
            question = question.text.strip()
            if question == '' or question in reference:
                continue
            data.append((question, answer, idx))
            reference[question] = (idx, answer)

data = pd.DataFrame(data, columns=['question', 'answer', 'answer_id'])

In [4]:
filtered = []
annotated = []
filter_annotated = []
tokenized_questions = []
lemma_filtered = []

for i, row in tqdm(data.iterrows()):
    doc = nlp(row['question'])
    tokenized_questions.append(' '.join([token.text.lower() for token in doc]))
    annotated.append(' '.join([token.text.lower() + ' ' + token.pos_ for token in doc]))
    filtered.append(' '.join([token.text.lower() for token in doc if token.is_alpha and not token.is_stop]))
    filter_annotated.append(' '.join([token.text.lower() + ' ' + token.pos_ for token in doc if token.is_alpha and not token.is_stop]))
    lemma_filtered.append(' '.join([token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop]))

data['question'] = tokenized_questions
data['filtered'] = filtered
data['annotated'] = annotated
data['filter_annotated'] = filter_annotated
data['lemma_filtered'] = filter_annotated
data.head()

,question,answer,answer_id,filtered,annotated,filter_annotated,lemma_filtered
0,"pode ser emitido cartão eletrónico da empresa ou de pessoa coletiva à s entidades titulares de cartão definitivo de identificação de pessoa coletiva ou entidade equiparada , emitido antes do dia 31/12/2008 ou inscritas anteriormente a essa data ?","Pode, desde que seja previamente pedido o Cartão da Empresa ou Cartão de Pessoa Coletiva, em suporte físico, que já contém o código de acesso ao cartão eletrónico.",1,pode emitido cartão eletrónico empresa pessoa coletiva s entidades titulares cartão definitivo identificação pessoa coletiva entidade equiparada emitido inscritas anteriormente a data,"pode AUX ser AUX emitido VERB cartão PROPN eletrónico PROPN da PROPN empresa PROPN ou CCONJ de ADP pessoa PROPN coletiva PROPN à ADP s DET entidades NOUN titulares ADJ de ADP cartão PROPN definitivo PROPN de ADP identificação PROPN de ADP pessoa PROPN coletiva PROPN ou CCONJ entidade NOUN equiparada ADJ , PUNCT emitido VERB antes ADV do ADP dia NOUN 31/12/2008 ADJ ou CCONJ inscritas VERB anteriormente ADV a ADP essa DET data NOUN ? PUNCT",pode AUX emitido VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN,pode AUX emitido VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN
1,"é possível emitir cartão eletrónico da empresa ou de pessoa coletiva à s entidades titulares de cartão definitivo de identificação de pessoa coletiva ou entidade equiparada , emitido antes do dia 31/12/2008 ou inscritas anteriormente a essa data ?","Pode, desde que seja previamente pedido o Cartão da Empresa ou Cartão de Pessoa Coletiva, em suporte físico, que já contém o código de acesso ao cartão eletrónico.",1,é emitir cartão eletrónico empresa pessoa coletiva s entidades titulares cartão definitivo identificação pessoa coletiva entidade equiparada emitido inscritas anteriormente a data,"é VERB possível ADJ emitir VERB cartão PROPN eletrónico PROPN da PROPN empresa PROPN ou CCONJ de ADP pessoa PROPN coletiva PROPN à ADP s DET entidades NOUN titulares ADJ de ADP cartão PROPN definitivo PROPN de ADP identificação PROPN de ADP pessoa PROPN coletiva PROPN ou CCONJ entidade NOUN equiparada ADJ , PUNCT emitido VERB antes ADV do ADP dia NOUN 31/12/2008 ADJ ou CCONJ inscritas VERB anteriormente ADV a ADP essa DET data NOUN ? PUNCT",é VERB emitir VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN,é VERB emitir VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN
2,"posso mandar fazer o cartão eletrónico da empresa ou de pessoa coletiva à s entidades titulares de cartão definitivo de identificação de pessoa coletiva ou entidade equiparada , emitido antes do dia 31/12/2008 ou inscritas antes d essa data ?","Pode, desde que seja previamente pedido o Cartão da Empresa ou Cartão de Pessoa Coletiva, em suporte físico, que já contém o código de acesso ao cartão eletrónico.",1,posso mandar o cartão eletrónico empresa pessoa coletiva s entidades titulares cartão definitivo identificação pessoa coletiva entidade equiparada emitido inscritas d data,"posso AUX mandar VERB fazer VERB o DET cartão PROPN ele

In [5]:
train, test = train_test_split(data, test_size=0.3)
train.sort_index(inplace=True)
test.sort_index(inplace=True)
print(f'{len(data)} -> {len(train)}, {len(test)}')
data.head()

2327 -> 1628, 699


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,question,answer,answer_id,filtered,annotated,filter_annotated,lemma_filtered
0,"pode ser emitido cartão eletrónico da empresa ou de pessoa coletiva à s entidades titulares de cartão definitivo de identificação de pessoa coletiva ou entidade equiparada , emitido antes do dia 31/12/2008 ou inscritas anteriormente a essa data ?","Pode, desde que seja previamente pedido o Cartão da Empresa ou Cartão de Pessoa Coletiva, em suporte físico, que já contém o código de acesso ao cartão eletrónico.",1,pode emitido cartão eletrónico empresa pessoa coletiva s entidades titulares cartão definitivo identificação pessoa coletiva entidade equiparada emitido inscritas anteriormente a data,"pode AUX ser AUX emitido VERB cartão PROPN eletrónico PROPN da PROPN empresa PROPN ou CCONJ de ADP pessoa PROPN coletiva PROPN à ADP s DET entidades NOUN titulares ADJ de ADP cartão PROPN definitivo PROPN de ADP identificação PROPN de ADP pessoa PROPN coletiva PROPN ou CCONJ entidade NOUN equiparada ADJ , PUNCT emitido VERB antes ADV do ADP dia NOUN 31/12/2008 ADJ ou CCONJ inscritas VERB anteriormente ADV a ADP essa DET data NOUN ? PUNCT",pode AUX emitido VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN,pode AUX emitido VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN
1,"é possível emitir cartão eletrónico da empresa ou de pessoa coletiva à s entidades titulares de cartão definitivo de identificação de pessoa coletiva ou entidade equiparada , emitido antes do dia 31/12/2008 ou inscritas anteriormente a essa data ?","Pode, desde que seja previamente pedido o Cartão da Empresa ou Cartão de Pessoa Coletiva, em suporte físico, que já contém o código de acesso ao cartão eletrónico.",1,é emitir cartão eletrónico empresa pessoa coletiva s entidades titulares cartão definitivo identificação pessoa coletiva entidade equiparada emitido inscritas anteriormente a data,"é VERB possível ADJ emitir VERB cartão PROPN eletrónico PROPN da PROPN empresa PROPN ou CCONJ de ADP pessoa PROPN coletiva PROPN à ADP s DET entidades NOUN titulares ADJ de ADP cartão PROPN definitivo PROPN de ADP identificação PROPN de ADP pessoa PROPN coletiva PROPN ou CCONJ entidade NOUN equiparada ADJ , PUNCT emitido VERB antes ADV do ADP dia NOUN 31/12/2008 ADJ ou CCONJ inscritas VERB anteriormente ADV a ADP essa DET data NOUN ? PUNCT",é VERB emitir VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN,é VERB emitir VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN
2,"posso mandar fazer o cartão eletrónico da empresa ou de pessoa coletiva à s entidades titulares de cartão definitivo de identificação de pessoa coletiva ou entidade equiparada , emitido antes do dia 31/12/2008 ou inscritas antes d essa data ?","Pode, desde que seja previamente pedido o Cartão da Empresa ou Cartão de Pessoa Coletiva, em suporte físico, que já contém o código de acesso ao cartão eletrónico.",1,posso mandar o cartão eletrónico empresa pessoa coletiva s entidades titulares cartão definitivo identificação pessoa coletiva entidade equiparada emitido inscritas d data,"posso AUX mandar VERB fazer VERB o DET cartão PROPN ele

In [6]:
# Make answers that don't show up in the train data appear as `answer_id = 0` in test
available_answer_ids = list(set(train['answer_id'].values))
for i, row in test.iterrows():
    if row['answer_id'] not in available_answer_ids:
        test.at[i, 'answer_id'] = 0
        print('No available id for row:', i)
test.head()

No available id for row: 112
No available id for row: 990
No available id for row: 991
No available id for row: 992
No available id for row: 993
No available id for row: 1382
No available id for row: 1383
No available id for row: 1384
No available id for row: 1385
No available id for row: 1442
No available id for row: 1443
No available id for row: 1444
No available id for row: 1445
No available id for row: 1685
No available id for row: 1686
No available id for row: 1687
No available id for row: 1688
No available id for row: 1865
No available id for row: 1866
No available id for row: 1867
No available id for row: 1868
No available id for row: 1897
No available id for row: 1898
No available id for row: 1899
No available id for row: 1900


,question,answer,answer_id,filtered,annotated,filter_annotated,lemma_filtered
0,"pode ser emitido cartão eletrónico da empresa ou de pessoa coletiva à s entidades titulares de cartão definitivo de identificação de pessoa coletiva ou entidade equiparada , emitido antes do dia 31/12/2008 ou inscritas anteriormente a essa data ?","Pode, desde que seja previamente pedido o Cartão da Empresa ou Cartão de Pessoa Coletiva, em suporte físico, que já contém o código de acesso ao cartão eletrónico.",1,pode emitido cartão eletrónico empresa pessoa coletiva s entidades titulares cartão definitivo identificação pessoa coletiva entidade equiparada emitido inscritas anteriormente a data,"pode AUX ser AUX emitido VERB cartão PROPN eletrónico PROPN da PROPN empresa PROPN ou CCONJ de ADP pessoa PROPN coletiva PROPN à ADP s DET entidades NOUN titulares ADJ de ADP cartão PROPN definitivo PROPN de ADP identificação PROPN de ADP pessoa PROPN coletiva PROPN ou CCONJ entidade NOUN equiparada ADJ , PUNCT emitido VERB antes ADV do ADP dia NOUN 31/12/2008 ADJ ou CCONJ inscritas VERB anteriormente ADV a ADP essa DET data NOUN ? PUNCT",pode AUX emitido VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN,pode AUX emitido VERB cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas VERB anteriormente ADV a ADP data NOUN
3,"quero emitir o cartão eletrónico da empresa ou de pessoa coletiva à s entidades titulares de cartão definitivo de identificação de pessoa coletiva ou entidade equiparada , emitido antes do dia 31/12/2008 ou inscritas antes d essa data . é possivel ?","Pode, desde que seja previamente pedido o Cartão da Empresa ou Cartão de Pessoa Coletiva, em suporte físico, que já contém o código de acesso ao cartão eletrónico.",1,quero emitir o cartão eletrónico empresa pessoa coletiva s entidades titulares cartão definitivo identificação pessoa coletiva entidade equiparada emitido inscritas d data é possivel,"quero VERB emitir VERB o DET cartão PROPN eletrónico PROPN da PROPN empresa PROPN ou CCONJ de ADP pessoa PROPN coletiva PROPN à ADP s DET entidades NOUN titulares ADJ de ADP cartão PROPN definitivo PROPN de ADP identificação PROPN de ADP pessoa PROPN coletiva PROPN ou CCONJ entidade NOUN equiparada ADJ , PUNCT emitido VERB antes ADV do ADP dia NOUN 31/12/2008 ADJ ou CCONJ inscritas NOUN antes ADV d ADP essa DET data NOUN . PUNCT é VERB possivel ADJ ? PUNCT",quero VERB emitir VERB o DET cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas NOUN d ADP data NOUN é VERB possivel ADJ,quero VERB emitir VERB o DET cartão PROPN eletrónico PROPN empresa PROPN pessoa PROPN coletiva PROPN s DET entidades NOUN titulares ADJ cartão PROPN definitivo PROPN identificação PROPN pessoa PROPN coletiva PROPN entidade NOUN equiparada ADJ emitido VERB inscritas NOUN d ADP data NOUN é VERB possivel ADJ
7,o cartão provisório de identificação de pessoa coletiva é pedido como ?,"O Cartão Provisório de Identificação de Pessoa Coletiva deixou de ser emitido, uma vez que as alterações legislativas ao regime jurídico do RNPC, ocorridas pelo Decreto-Lei n.º 247-B/2008, de 30 de Dezembro, não contemplam a sua emissão.\n Atualmente existe apenas o Cartão da Empresa e o Cartão de Pessoa Coletiva que são emitidos para entidades definitivamente registadas ou inscritas.",2,o cartão provisório identificação pessoa coletiva pedido,o DET cartão PROPN provisório PROPN de ADP identificação PROPN de ADP pess

## Experiments
* Word-level distance measures: nltk.metrics.distance.*
* scale distance by the length of the sentence (ok to have few differences in long sentences)
* Annotate the initial sentence with spaCy (POS tags, etc)
* remove stopwords, punctuation, and everything that is not alphabetical

In [7]:
def query(q, corpus, distance, threshold=10, column='question'):
    """Return answer_id with the most similar question"""
    distances = [distance(q, row[column]) for i, row in corpus.iterrows()]
    best_match = np.argmin([distances])
    
    if distances[best_match] >= threshold:
        return 0, distances[best_match]
    return corpus.iloc[best_match]['answer_id'], distances[best_match]

In [18]:
distance_thresholds = [
    (lambda a, b: jaccard_distance(set(a.split()), set(b.split())), 0.6),
    (lambda a, b: jaccard_distance(set(a.split()), set(b.split())), 0.7),
    (lambda a, b: jaccard_distance(set(a.split()), set(b.split())), 0.8),
    (lambda a, b: masi_distance(set(a.split()), set(b.split())), 0.6),    
    (lambda a, b: masi_distance(set(a.split()), set(b.split())), 0.7),    
    (lambda a, b: masi_distance(set(a.split()), set(b.split())), 0.8),    
    (lambda a, b: edit_distance(a.split(), b.split()), 8),
    (lambda a, b: edit_distance(a.split(), b.split()), 16),
    (lambda a, b: edit_distance(a.split(), b.split()), 24),
]
columns = [
    'question',
    'filtered',
    'annotated',
    'filter_annotated',
    'lemma_filtered',
]

## Evaluate

In [17]:
def evaluate(column, distance, threshold, verbose=True):
    pred, label, distances = [], [], []
    for i, row in tqdm(test.iterrows(), disable=not verbose):
        ans_id, dist = query(
            row[column],
            corpus=train,
            distance=distance,
            threshold=threshold,
            column=column,
        )
        pred.append(int(ans_id))
        distances.append(dist)
        label.append(int(row['answer_id']))

    acc = accuracy_score(label, pred) * 100
    print(f'ACC: {acc:.2f}')
    return pred, label, distances

In [19]:
for distance_measure, threshold in distance_thresholds:
    for c in columns:
        print(f'Evaluating dist: {distance_measure}, threshold: {threshold}, column: {c}', end='...')
        pred, label, distances = evaluate(column=c, distance=distance_measure, threshold=threshold, verbose=False)
        # for p, l, d  in zip(pred, label, distances):
        #     print(f'pred: {p}, label: {l}, dist: {d}')

Evaluating dist: <function <lambda> at 0x119fe0158>, threshold: 0.6, column: question...ACC: 68.53
Evaluating dist: <function <lambda> at 0x119fe0158>, threshold: 0.6, column: filtered...ACC: 66.67
Evaluating dist: <function <lambda> at 0x119fe0158>, threshold: 0.6, column: annotated...ACC: 66.09
Evaluating dist: <function <lambda> at 0x119fe0158>, threshold: 0.6, column: filter_annotated...ACC: 65.67
Evaluating dist: <function <lambda> at 0x119fe0158>, threshold: 0.6, column: lemma_filtered...ACC: 65.67
Evaluating dist: <function <lambda> at 0x119fe0bf8>, threshold: 0.7, column: question...ACC: 69.96
Evaluating dist: <function <lambda> at 0x119fe0bf8>, threshold: 0.7, column: filtered...ACC: 70.96
Evaluating dist: <function <lambda> at 0x119fe0bf8>, threshold: 0.7, column: annotated...ACC: 66.24
Evaluating dist: <function <lambda> at 0x119fe0bf8>, threshold: 0.7, column: filter_annotated...ACC: 65.24
Evaluating dist: <function <lambda> at 0x119fe0bf8>, threshold: 0.7, column: lemma_fi